In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Disabling warnings
import warnings
warnings.simplefilter("ignore")

In [3]:
# This function is taken from https://www.kaggle.com/rinnqd/reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
       
        1. Iterate over every column
        2. Determine if the column is numeric
        3. Determine if the column can be represented by an integer
        4. Find the min and the max value
        5. Determine and apply the smallest datatype that can fit the range of values

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
POS_CASH_balance = import_data("POS_CASH_balance.csv")

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 171.69 MB
Decreased by 71.9%


In [5]:
df_POS_CASH_balance = POS_CASH_balance.copy()

## Summary Info
- POS_CASH_balance
    - RangeIndex: 10,001,358 entries, 0 to 10,001,357
    - Data columns (total 8 columns)
    - float64(2), int64(5), object(1)

In [6]:
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None

## POS_CASH Balance

In [7]:
df_POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [8]:
total_nan = df_POS_CASH_balance.isnull().sum().sort_values(ascending = False)
percent_nan = (df_POS_CASH_balance.isnull().sum()/df_POS_CASH_balance.isnull().count()*100).sort_values(ascending = False)
missing_pos_cash_data  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_pos_cash_data

,Total_nan,Percent_nan
CNT_INSTALMENT_FUTURE,26087,0.260835
CNT_INSTALMENT,26071,0.260675
SK_DPD_DEF,0,0.000000
SK_DPD,0,0.000000
NAME_CONTRACT_STATUS,0,0.000000
MONTHS_BALANCE,0,0.000000
SK_ID_CURR,0,0.000000
SK_ID_PREV,0,0.000000


### Replacing XNA and one-hot encoding

In [9]:
df_POS_CASH_balance.NAME_CONTRACT_STATUS.replace('XNA',np.nan,inplace=True)

In [10]:
# for i in df_POS_CASH_balance.columns:
#     print(i)
#     print(df_POS_CASH_balance[i][df_POS_CASH_balance[i]=="XNA"].count())
#     print(df_POS_CASH_balance[i][df_POS_CASH_balance[i]=="XAP"].count())

### New Feature
- CNT_INSTALMENT_DIFF = CNT_INSTALMENT – CNT_INSTALMENT_FUTURE #Remaining installments

In [11]:
df_POS_CASH_balance['CNT_INSTALMENT_DIFF'] = df_POS_CASH_balance['CNT_INSTALMENT'] - df_POS_CASH_balance['CNT_INSTALMENT_FUTURE']

In [12]:
total_nan = df_POS_CASH_balance.isnull().sum().sort_values(ascending = False)
percent_nan = (df_POS_CASH_balance.isnull().sum()/df_POS_CASH_balance.isnull().count()*100).sort_values(ascending = False)
missing_pos_cash_data  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_pos_cash_data

,Total_nan,Percent_nan
CNT_INSTALMENT_DIFF,26184,0.261804
CNT_INSTALMENT_FUTURE,26087,0.260835
CNT_INSTALMENT,26071,0.260675
NAME_CONTRACT_STATUS,2,0.000020
SK_DPD_DEF,0,0.000000
SK_DPD,0,0.000000
MONTHS_BALANCE,0,0.000000
SK_ID_CURR,0,0.000000
SK_ID_PREV,0,0.000000


In [13]:
df_POS_CASH_balance.NAME_CONTRACT_STATUS = df_POS_CASH_balance.NAME_CONTRACT_STATUS.astype('category')
df_POS_CASH_balance = pd.get_dummies(df_POS_CASH_balance, columns = ['NAME_CONTRACT_STATUS'])
df_POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT_DIFF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195,182943,-31,48.0,45.0,0,0,3.0,1,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1.0,1,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,3.0,1,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,6.0,1,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1.0,1,0,0,0,0,0,0,0,0


### Imputing NaN Values (optional)
- We can use YCImpute package to impute NaN values. EM algorithm gives result in a shorter time. KNN is time consuming and Iterforest gives error. 
- A function is formed for imputation. We need to convert the dataframe into an array and the values become float. We add lines to keep the datatypes same as before the method.
- __Note__: Imputing NaN values with this algorithm decreases the AUC score compared to data set without imputaion. Different algorithms and alternatively sklearn tools can be used and compared but it is very time consuming with large data. 

In [14]:
# from ycimpute.imputer import iterforest
# from ycimpute.imputer import EM
# from ycimpute.imputer import knnimput

# def nan_imputer(df):
#     int_columns = df.select_dtypes(include='int64').columns
#     cat_columns = df.select_dtypes(include='uint8').columns
#     var_names = df.columns
#     np_df = np.array(df)
#     df = EM().complete(np_df)
#     df = pd.DataFrame(df, columns = var_names)
#     df[int_columns] = df[int_columns].astype('int64')
#     df[cat_columns] = df[cat_columns].astype('uint8')
#     return df

In [15]:
# df_bureau = nan_imputer(df_POS_CASH_balance)

In [16]:
# df_bureau.isnull().sum()

In [17]:
df_POS_CASH_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 17 columns):
SK_ID_PREV                                    int32
SK_ID_CURR                                    int32
MONTHS_BALANCE                                int8
CNT_INSTALMENT                                float16
CNT_INSTALMENT_FUTURE                         float16
SK_DPD                                        int16
SK_DPD_DEF                                    int16
CNT_INSTALMENT_DIFF                           float16
NAME_CONTRACT_STATUS_Active                   uint8
NAME_CONTRACT_STATUS_Amortized debt           uint8
NAME_CONTRACT_STATUS_Approved                 uint8
NAME_CONTRACT_STATUS_Canceled                 uint8
NAME_CONTRACT_STATUS_Completed                uint8
NAME_CONTRACT_STATUS_Demand                   uint8
NAME_CONTRACT_STATUS_Returned to the store    uint8
NAME_CONTRACT_STATUS_Signed                   uint8
NAME_CONTRACT_STATUS_XNA                      ui

## Aggregation
- We use two functions for aggregation, one for numerical features and one for categorical features. Because, it is not reasonable to calculate some of the statistical values for categorical variables. 
- We use groupby method for aggregations in the functions. 
- We aggregate the observations based on SK_ID_CURR since we may have more than one previous credit for a current applicant. 
- @willkoehrsen
- https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering

In [18]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum, std) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes(exclude=['uint8'])
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum', 'var']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [19]:
def agg_categorical(df, group_var, df_name):
    """Aggregates the encoded categorical values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all labeled categorical columns. Each instance of the grouping variable 
            will have the statistics (mean, min, max, sum, std) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    categorical_df = df.select_dtypes(include=['uint8'])
    categorical_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = categorical_df.groupby(group_var).agg(['count', 'sum', 'mean']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

### POS_CASH Balance Aggregation

In [20]:
pos_cash_agg_num = agg_numeric(df_POS_CASH_balance.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR', df_name = 'pos_cash')
pos_cash_agg_num.head()

,SK_ID_CURR,pos_cash_MONTHS_BALANCE_count,pos_cash_MONTHS_BALANCE_mean,pos_cash_MONTHS_BALANCE_max,pos_cash_MONTHS_BALANCE_min,pos_cash_MONTHS_BALANCE_sum,pos_cash_MONTHS_BALANCE_var,pos_cash_CNT_INSTALMENT_count,pos_cash_CNT_INSTALMENT_mean,pos_cash_CNT_INSTALMENT_max,...,pos_cash_SK_DPD_DEF_max,pos_cash_SK_DPD_DEF_min,pos_cash_SK_DPD_DEF_sum,pos_cash_SK_DPD_DEF_var,pos_cash_CNT_INSTALMENT_DIFF_count,pos_cash_CNT_INSTALMENT_DIFF_mean,pos_cash_CNT_INSTALMENT_DIFF_max,pos_cash_CNT_INSTALMENT_DIFF_min,pos_cash_CNT_INSTALMENT_DIFF_sum,pos_cash_CNT_INSTALMENT_DIFF_var
0,100001,9,-72.555556,-53,-96,-653.0,435.277778,9,4.000000,4.0,...,7,0,7.0,5.444444,9,2.554688,4.0,0.0,23.0,2.027344
1,100002,19,-10.000000,-1,-19,-190.0,31.666667,19,24.000000,24.0,...,0,0,0.0,0.000000,19,9.000000,18.0,0.0,171.0,31.671875
2,100003,28,-43.785714,-18,-77,-1226.0,607.137566,28,10.109375,12.0,...,0,0,0.0,0.000000,28,4.320312,11.0,0.0,121.0,9.265625
3,100004,4,-25.500000,-24,-27,-102.0,1.666667,4,3.750000,4.0,...,0,0,0.0,0.000000,4,1.500000,3.0,0.0,6.0,1.666992
4,100005,11,-20.000000,-15,-25,-220.0,11.000000,10,11.703125,12.0,...,0,0,0.0,0.000000,10,4.500000,9.0,0.0,45.0,9.164062


In [21]:
pos_cash_agg_cat = agg_categorical(df_POS_CASH_balance.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR', df_name = 'pos_cash')
pos_cash_agg_cat.head()

,SK_ID_CURR,pos_cash_NAME_CONTRACT_STATUS_Active_count,pos_cash_NAME_CONTRACT_STATUS_Active_sum,pos_cash_NAME_CONTRACT_STATUS_Active_mean,pos_cash_NAME_CONTRACT_STATUS_Amortized debt_count,pos_cash_NAME_CONTRACT_STATUS_Amortized debt_sum,pos_cash_NAME_CONTRACT_STATUS_Amortized debt_mean,pos_cash_NAME_CONTRACT_STATUS_Approved_count,pos_cash_NAME_CONTRACT_STATUS_Approved_sum,pos_cash_NAME_CONTRACT_STATUS_Approved_mean,...,pos_cash_NAME_CONTRACT_STATUS_Demand_mean,pos_cash_NAME_CONTRACT_STATUS_Returned to the store_count,pos_cash_NAME_CONTRACT_STATUS_Returned to the store_sum,pos_cash_NAME_CONTRACT_STATUS_Returned to the store_mean,pos_cash_NAME_CONTRACT_STATUS_Signed_count,pos_cash_NAME_CONTRACT_STATUS_Signed_sum,pos_cash_NAME_CONTRACT_STATUS_Signed_mean,pos_cash_NAME_CONTRACT_STATUS_XNA_count,pos_cash_NAME_CONTRACT_STATUS_XNA_sum,pos_cash_NAME_CONTRACT_STATUS_XNA_mean
0,100001,9,7.0,0.777778,9,0,0.0,9,0,0.0,...,0.0,9,0,0.0,9,0,0.000000,9,0,0
1,100002,19,19.0,1.000000,19,0,0.0,19,0,0.0,...,0.0,19,0,0.0,19,0,0.000000,19,0,0
2,100003,28,26.0,0.928571,28,0,0.0,28,0,0.0,...,0.0,28,0,0.0,28,0,0.000000,28,0,0
3,100004,4,3.0,0.750000,4,0,0.0,4,0,0.0,...,0.0,4,0,0.0,4,0,0.000000,4,0,0
4,100005,11,9.0,0.818182,11,0,0.0,11,0,0.0,...,0.0,11,0,0.0,11,1,0.090909,11,0,0


In [22]:
total_nan = pos_cash_agg_num.isnull().sum().sort_values(ascending = False)
percent_nan = (pos_cash_agg_num.isnull().sum()/pos_cash_agg_num.isnull().count()*100).sort_values(ascending = False)
missing_pos_cash_data  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_pos_cash_data.head(16)

,Total_nan,Percent_nan
pos_cash_CNT_INSTALMENT_DIFF_var,394,0.116827
pos_cash_CNT_INSTALMENT_var,394,0.116827
pos_cash_CNT_INSTALMENT_FUTURE_var,394,0.116827
pos_cash_MONTHS_BALANCE_var,372,0.110303
pos_cash_SK_DPD_DEF_var,372,0.110303
pos_cash_SK_DPD_var,372,0.110303
pos_cash_CNT_INSTALMENT_DIFF_max,28,0.008302
pos_cash_CNT_INSTALMENT_DIFF_mean,28,0.008302
pos_cash_CNT_INSTALMENT_mean,28,0.008302
pos_cash_CNT_INSTALMENT_max,28,0.008302


In [23]:
# pos_cash_agg_cat.to_csv('pos_cash_agg_cat.csv', index=False)

In [24]:
# pos_cash_agg_num.to_csv('pos_cash_agg_num.csv', index=False)

In [ ]:
# pd.reset_option("display.max_rows")
# pd.reset_option("display.max_columns")
# pd.get_option("display.max_rows")